In [16]:
import os
import sys
import pandas as pd
import yfinance as yf
import numpy as np
sys.path.append(os.path.join('Momentum_additional_files'))

from config import *
from helper import *
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

# price_data_df = collecting_data(START_DATE, END_DATE, DATA_ADJ_CLOSE_LOC, DATA_DIV_LOC)
# relative_df = get_relative(price_data_df)

In [14]:
start_date, end_date, file_price_loc, file_div_loc = START_DATE, END_DATE, DATA_OPEN_LOC, DATA_DIV_LOC
def fetch_sp500_companies():
    # Fetch the list of S&P 500 companies from Wikipedia
    url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Parse the table containing the S&P 500 companies
    table = soup.find('table', {'class': 'wikitable'})
    df = pd.read_html(str(table))[0]

    # Return the list of company symbols
    return df['Symbol'].tolist()

def get_sp500_companies_active_in_year(start_date, end_date):
    # Get the current S&P 500 companies
    sp500_companies = fetch_sp500_companies()
    
    active_companies = []
    for symbol in sp500_companies:
        try:
            # Get historical data for the company
            company = yf.Ticker(symbol)
            historical_data = company.history(start=start_date, end=end_date)
            
            # Check if data exists for that year
            if not historical_data.empty:
                active_companies.append(symbol)
        except Exception as e:
            # print(f"Could not retrieve data for {symbol}: {e}")
            pass

    return active_companies


# Get the list of S&P 500 companies for the year 2004
companies_2004 = get_sp500_companies_active_in_year(start_date, end_date)
price_data = yf.download(companies_2004, start=start_date, end=end_date, interval='1mo', actions =True)[['Adj Close','Dividends']]
num_columns_with_nan = price_data.isnull().any().sum()
print(f'The number of columns that have nan values: {num_columns_with_nan}, and are been dropped.')
price_data = price_data.dropna(axis=1)
price_data[STOCK_TIME].to_csv(file_price_loc)

$BRK.B: possibly delisted; no timezone found
$BF.B: possibly delisted; no price data found  (1d 2004-01-01 -> 2024-10-01)
[*********************100%***********************]  501 of 501 completed

1 Failed download:
['AMTM']: IndexError('index 0 is out of bounds for axis 0 with size 0')


The number of columns that have nan values: 237, and are been dropped.


In [19]:
for i in range(len(price_data) - FORMATION_PERIOD_MONTHS):
    block_12 = block[:-1]
    print(block_12)
    break
    date_index = block.index[-1]
    diff = (block_12[STOCK_TIME].iloc[-1] + block_12['Dividends'].sum(axis=0) - block_12[STOCK_TIME].iloc[0]) / block_12[STOCK_TIME].iloc[0]
    row_df = pd.DataFrame(diff).T
    row_df.index = [date_index]
    relative_df = pd.concat([relative_df, row_df])

NameError: name 'block' is not defined

In [ ]:
for i in range(len(df) - FORMATION_PERIOD_MONTHS ):
    block = df.iloc[i:i + FORMATION_PERIOD_MONTHS + 1] # Add row to get the date index
    block_12 = block[:-1]
    date_index = block.index[-1]
    diff = (block_12[STOCK_TIME].iloc[-1] + block_12['Dividends'].sum(axis=0) - block_12[STOCK_TIME].iloc[0]) / block_12[STOCK_TIME].iloc[0]
    row_df = pd.DataFrame(diff).T
    row_df.index = [date_index]
    relative_df = pd.concat([relative_df, row_df])

In [ ]:
# lag close
# print lag
# add technical indicators lagged
# fix size window (days instead months?)
# predict

In [10]:
i = 1
df.iloc[i: i + 12 + 1]

NameError: name 'df' is not defined

In [ ]:
def get_relative(df):
    relative_df = pd.DataFrame(columns=df[STOCK_TIME].columns.values)
    
    for i in range(len(df) - FORMATION_PERIOD_MONTHS ):
        block = df.iloc[i:i + FORMATION_PERIOD_MONTHS + 1] # Add row to get the date index
        block_12 = block[:-1]
        date_index = block.index[-1]
        diff = (block_12[STOCK_TIME].iloc[-1] + block_12['Dividends'].sum(axis=0) - block_12[STOCK_TIME].iloc[0]) / block_12[STOCK_TIME].iloc[0]
        row_df = pd.DataFrame(diff).T
        row_df.index = [date_index]
        relative_df = pd.concat([relative_df, row_df])
    return relative_df


In [5]:

import pandas as pd
import numpy as np

# Generate synthetic stock data
def create_dummy_stock_data():
    np.random.seed(42)
    n_days = 1500  # ~6 years of data
    dates = pd.date_range(start="2015-01-01", periods=n_days, freq="B")  # Business days
    close_prices = np.cumsum(np.random.randn(n_days) * 2 + 100)  # Random walk
    volumes = np.random.randint(100, 1000, size=n_days)  # Random volume data

    # Build DataFrame
    df = pd.DataFrame({
        "Date": dates,
        "Close": close_prices,
        "Volume": volumes
    })
    return df

# Save the dummy data to a CSV
df_dummy = create_dummy_stock_data()

In [8]:
i=0
FORMATION_PERIOD_MONTHS=12

[i:i + FORMATION_PERIOD_MONTHS + 1]

,Date,Close,Volume
0,2015-01-01,100.993428,258
1,2015-01-02,200.716900,287
2,2015-01-05,302.012277,159
3,2015-01-06,405.058336,916
4,2015-01-07,504.590030,418
5,2015-01-08,604.121756,977
6,2015-01-09,707.280181,310
7,2015-01-12,808.815051,505
8,2015-01-13,907.876102,763
9,2015-01-14,1008.961222,208


In [18]:
import pandas as pd
import numpy as np
import torch
from torch import nn, optim
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from ta.momentum import RSIIndicator
from ta.volatility import BollingerBands
from ta.trend import SMAIndicator
from ta.volume import OnBalanceVolumeIndicator

# 1. Compute Technical Indicators
def compute_technical_indicators(df):
    df['RSI'] = RSIIndicator(close=df['Close'], window=14).rsi()
    df['MA'] = SMAIndicator(close=df['Close'], window=20).sma_indicator()
    bb = BollingerBands(close=df['Close'], window=20)
    df['BB_High'] = bb.bollinger_hband()
    df['BB_Low'] = bb.bollinger_lband()
    df['OBV'] = OnBalanceVolumeIndicator(close=df['Close'], volume=df['Volume']).on_balance_volume()
    df = df.dropna()  # Drop rows with NaN values
    return df

# 2. Generate Labels: Buy (1) or Short (0)
def generate_labels(df):
    df['Future_Close'] = df['Close'].shift(-90)  # 3 months ahead
    df['Signal'] = np.where(df['Future_Close'] > df['Close'], 1, 0)
    df = df.dropna()  # Drop rows with NaN values
    return df

# 3. Preprocess Data
def preprocess_data(df):
    scaler = MinMaxScaler()
    features = ['RSI', 'MA', 'BB_High', 'BB_Low', 'OBV', 'Close', 'Volume']
    df[features] = scaler.fit_transform(df[features])
    return df, scaler

# 4. Dataset and DataLoader
class StockDataset(torch.utils.data.Dataset):
    def __init__(self, data, lookback=252):
        self.data = data
        self.lookback = lookback
    
    def __len__(self):
        return len(self.data) - self.lookback
    
    def __getitem__(self, idx):
        x = self.data.iloc[idx:idx + self.lookback, :-1].values
        y = self.data.iloc[idx + self.lookback, -1]
        return torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.long)

# 5. Define the Model
class StockPredictor(nn.Module):
    def __init__(self, input_dim):
        super(StockPredictor, self).__init__()
        self.lstm = nn.LSTM(input_dim, 64, batch_first=True)
        self.fc = nn.Sequential(
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 2)  # 2 classes: Buy or Short
        )
    
    def forward(self, x):
        _, (hn, _) = self.lstm(x)
        out = self.fc(hn[-1])
        return out

# 6. Training and Evaluation
def train_model(model, train_loader, val_loader, epochs=20, lr=0.001):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        for x, y in train_loader:
            optimizer.zero_grad()
            outputs = model(x)
            loss = criterion(outputs, y)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        val_loss = evaluate_model(model, val_loader, criterion)
        print(f"Epoch {epoch + 1}/{epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

def evaluate_model(model, val_loader, criterion):
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for x, y in val_loader:
            outputs = model(x)
            loss = criterion(outputs, y)
            val_loss += loss.item()
    return val_loss / len(val_loader)

# 7. Main Workflow
def main():
    # Load data
    df = pd.read_csv('your_stock_data.csv')  # Replace with your data file
    df = compute_technical_indicators(df)
    df = generate_labels(df)
    df, scaler = preprocess_data(df)
    
    # Split data
    train_data, test_data = train_test_split(df, test_size=0.2, shuffle=False)
    train_dataset = StockDataset(train_data)
    test_dataset = StockDataset(test_data)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)
    
    # Initialize and train the model
    input_dim = len(train_data.columns) - 1  # Exclude label column
    model = StockPredictor(input_dim)
    train_model(model, train_loader, val_loader)

if __name__ == "__main__":
    main()


ModuleNotFoundError: No module named 'torch'